In [ ]:
from glob import glob

import sys
sys.path.append("/home/dzilov/Dropbox/gepymics/")
from gepymics import fasta_read
from gepymics import reverse_complement
from collections import defaultdict

In [ ]:
# DnaA

anno_files = glob("/mnt/projects/zilov/data/mycoplasma/annotation/*gff")
nucl_prot_files = glob("/mnt/projects/zilov/data/mycoplasma/genomes/*fna")

In [ ]:
dnaa_ids = defaultdict()

for i, file in enumerate(anno_files):
    with open(file) as fh:
        for line in fh:
            if line.startswith("#"):
                continue            
            if "dnaA" in line:
                line = line.split("\t")
                coordinates = [int(line[3]), int(line[4]), line[6]]
                dnaa_ids[line[0]] = coordinates
                i += 1
print(i)

In [ ]:
dnaa_fasta = {}

i = 0
for file in nucl_prot_files:
    fasta = fasta_read(file)
    for nid, coordinates in dnaa_ids.items():
        if nid in fasta.keys():
            if coordinates[-1] == "-":
                dnaa_fasta[nid] = reverse_complement(fasta[nid][coordinates[0]-1:coordinates[1]-1])
            else:
                dnaa_fasta[nid] = fasta[nid][coordinates[0]-1:coordinates[1]-1]
            i += 1
            
print(i)

In [ ]:
dnaa_file = "/media/eternus1/projects/zilov/data/mycoplasma/genes_fasta_to_msa/dnaa.fasta"

with open(dnaa_file, "w") as fw:
    for h, s in dnaa_fasta.items():
        fw.write(f">{h}\n{s}\n")

In [ ]:
consensus_file = "/media/eternus1/projects/zilov/data/mycoplasma/genes_fasta_to_msa/lcl_mod_ref-NZ_JNJG01000004.1 and 51 other sequences.aln"

In [ ]:
longest_atgc = ""
with open(consensus_file) as fh:
    score = 0
    max_score = 0
    miss = 0
    for line in fh:
        if not line.startswith(">"):
            for s in line:
                if score > max_score:
                    max_score = score
                if s in ["A", "T", "G", "C"]:
                    if score < 0:
                        score = 0
                    score += 1
                    print(s, score)
                elif s in ["W", "K", "N", "R", "M", "Y", "S", "B", "D", "H", "V"]:
                    score -= 1
                    print(s, score)
                else: 
                    score = 0

In [ ]:
max_score

In [ ]:
cds_fasta = glob("/home/dzilov/data_zilov/data/mycoplasma/*/*/")

In [ ]:
from glob import glob
import os
from collections import defaultdict

def get_organism(gff):
    genome_dir = os.path.dirname(gff)
    report = glob(f'{genome_dir}/*.gbff')
    
    with open(report[0]) as fh:
        for line in fh:
            if line.startswith('SOURCE'):
                organism = line.split('SOURCE')[-1].strip()
    return organism

def get_pid(gff, protein_name):
    with open(gff) as fh:
        for line in fh:
            if protein_name.lower() in line.lower():
                pid = line.split("\t")[8].split("Name=")[-1].split(";")[0]
                return pid
                
def fasta_read(path_to_fasta_file):
    "Returns dictionary header:sequence"
    fasta = {}
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header] = "".join(seq)
                header = line[1:].split()[0]
                seq = []
            else:
                seq.append(line)
        if header:
            fasta[header] = "".join(seq)
    return fasta

In [ ]:
pids2bact = {} 
for file in anno_files:
    if get_pid(file, 'chromosomal replication initiator protein DnaA'): 
        sco = 'chromosomal replication initiator protein DnaA'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    elif get_pid(file, 'chromosomal replication initiat'):
        sco = 'chromosomal replication initiat'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    elif get_pid(file, 'chromosomal replication protein'):
        sco = 'chromosomal replication protein'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    else:
        not_found += 1
    pids2bact[prot2bact[0]] = prot2bact[1]
    
print(found, not_found)

In [ ]:
## sco num_bact n_missed uniq_prots prots_in_multifasta multifasta_file
summary_table = []

anno_files = glob("/home/dzilov/data_zilov/data/mycoplasma/*/*/*.gff")

print(len(anno_files), 'genomes to analyze')

# read all cds fastas
all_fastas = []
for i in glob("/home/dzilov/data_zilov/data/mycoplasma/*/*/*cds_from_genomic.fna"):
    fasta = fasta_read(i)
    all_fastas.append(fasta)
    
# build multifasta for each sco by all bacteria


multifasta_file = f"/home/dzilov/data_zilov/data/mycoplasma/clusters/multifasta/dnaa.fasta"

#     if os.path.exists(multifasta_file) and os.path.getsize(multifasta_file) != 0:
#         print(f"Multifasta for {sco} already written, continue...")
#         continue


pids2bact = {} 
n_missed = 0
for file in anno_files:
    if get_pid(file, 'chromosomal replication initiator protein DnaA'): 
        sco = 'chromosomal replication initiator protein DnaA'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    elif get_pid(file, 'chromosomal replication initiat'):
        sco = 'chromosomal replication initiat'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    elif get_pid(file, 'chromosomal replication protein'):
        sco = 'chromosomal replication protein'
        prot2bact = [get_pid(file, sco), get_organism(file)]
    else:
        n_missed += 1
        print(f"MISSED {sco} in {file}, organism: {prot2bact[1]}")
    pids2bact[prot2bact[0]] = prot2bact[1]


print(f'{len(pids2bact)} uniq prots in {sco}')

multifasta = {}
for pid, organism in pids2bact.items():
    for prots in all_fastas:
        for header, seq in prots.items():
            if pid in header:
                short_organism = "_".join(organism.split(",")[0].split())
                multifasta[f"{pid}_{short_organism}"]=seq



with open(multifasta_file, "w") as fw:
    for h, s in multifasta.items():
        fw.write(f">{h}\n{s}\n")

print(len(multifasta.keys()), f'prots added to multifasta {multifasta_file}')

sco_summary = [sco, str(len(all_fastas)), str(n_missed), str(len(pids2bact)), multifasta_file]
summary_table.append(sco_summary)

101 prots added to multifasta /home/dzilov/data_zilov/data/mycoplasma/clusters/multifasta/dnaa.fasta


In [ ]:
def msa_to_consensus(fasta_msa, only_n_consensus=False, compare_treshold=.3,):

    consensus_table = defaultdict()
    
    fasta_msa = fasta_read(fasta_msa)

    for v in fasta_msa.values():
        for i in range(len(v)):
            consensus_table[i] = {'A': 0, 'C': 0, 'G': 0, 'T': 0, '-': 0, 'N': 0}
        break
    
    for k, v in fasta_msa.items():
        for i, n in enumerate(v):
            consensus_table[i][n] += 1

    consensus = []

    for k, v in consensus_table.items(): 
        few_n = []

        for n, c in v.items():
            if c == len(fasta_msa):
                few_n.append(n)
            else:
                if c != 0:
                    if n == '-':
                        if c / len(fasta_msa) > 0.7:
                            few_n = []
                    else:
                        if c / len(fasta_msa) > compare_treshold:
                            few_n.append(n)
                            
        if len(few_n) == 0:
            continue
        elif len(few_n) == 1:
            if '-' in few_n or 'N' in few_n:
                consensus.append('N')
            else:
                consensus.append(few_n[0])
        else:
            if only_n_consensus:
                consensus.append('N')
            else:
                if ('A' in few_n and 'T' in few_n and 'G' in few_n and 'C' in few_n) or ('-' in few_n or 'N' in few_n):
                    consensus.append('N')
                elif 'A' in few_n and 'G' in few_n:
                    if 'C' in few_n:
                        consensus.append('V')
                    elif 'T' in few_n:
                        consensus.append('D')
                    else:
                        consensus.append('R')
                elif 'A' in few_n and 'T' in few_n:
                    if 'C' in few_n:
                        consensus.append('H')
                    else:
                        consensus.append('W')
                elif 'G' in few_n and 'C' in few_n:
                    if 'T' in few_n:
                        consensus.append('B')
                    else:
                        consensus.append('S')
                elif 'G' in few_n and 'T' in few_n:
                    consensus.append('K')
                elif 'C' in few_n and 'T' in few_n:
                    consensus.append('Y')
                elif 'A' in few_n and 'C' in few_n:
                    consensus.append('M')

    return ''.join(consensus)